In [17]:
import cv2
import itertools
import numpy as np
import mediapipe as mp
import pandas as pd

In [18]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    refine_landmarks=True,
    static_image_mode=True,
    max_num_faces = 1,
)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

for i in range(100):
    image = cv2.imread("./dataset/train/img/%s.png"%i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    globals()['height'+str(i)] = image.shape[0]
    
    results = face_mesh.process(image)
    
    for single_face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list = single_face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec = drawing_spec,            
        )
    
    INDEX = mp_face_mesh.FACEMESH_LIPS
    INDEX_LIST = list(itertools.chain(*INDEX))
    
    globals()['facemesh'+str(i)] = np.zeros((len(INDEX_LIST), 2))
    
    for single_face_landmarks in results.multi_face_landmarks:
        for j, k in enumerate(INDEX_LIST):
            coordinates = single_face_landmarks.landmark[k]
            globals()['facemesh'+str(i)][j][0] = coordinates.x * image.shape[1]
            globals()['facemesh'+str(i)][j][1] = coordinates.y * image.shape[0]

In [19]:
for i in range(100):
    land = np.zeros((len(facemesh0),2))
    
    for j in range(len(facemesh0)):
        land[j][0] = globals()['facemesh'+str(i)][j][0]
        land[j][1] = globals()['facemesh'+str(i)][j][1]
        
    x = land[:, 0]
    y = land[:, 1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))
    
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [20]:
train_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        train_x[i][j] = globals()['norm'+str(i)][j]

In [21]:
for i in range(100):
    image = cv2.imread("./dataset/test/img/%s.png"%i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    globals()['height'+str(i)] = image.shape[0]
    
    results = face_mesh.process(image)
    
    for single_face_landmark in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list = single_face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec = drawing_spec,            
        )
    
    INDEX = mp_face_mesh.FACEMESH_LIPS
    INDEX_LIST = list(itertools.chain(*INDEX))
    
    globals()['facemesh'+str(i)] = np.zeros((len(INDEX_LIST), 2))
    
    for single_face_landmarks in results.multi_face_landmarks:
        for j, k in enumerate(INDEX_LIST):
            coordinates = single_face_landmarks.landmark[k]
            globals()['facemesh'+str(i)][j][0] = coordinates.x * image.shape[1]
            globals()['facemesh'+str(i)][j][1] = coordinates.y * image.shape[0]

In [22]:
for i in range(100):
    land = np.zeros((len(facemesh0),2))
    
    for j in range(len(facemesh0)):
        land[j][0] = globals()['facemesh'+str(i)][j][0]
        land[j][1] = globals()['facemesh'+str(i)][j][1]
        
    x = land[:, 0]
    y = land[:, 1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))
    
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [23]:
test_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        test_x[i][j] = globals()['norm'+str(i)][j]

In [24]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 30:]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 30:]

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)

LinearRegression()

In [26]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)
mae

0.14148936157145642

In [27]:
pred = pd.DataFrame(y_pred)
pred.columns = ["mouthSize","mouthPosition","mouthPronounced","lipSize"]
pred

,mouthSize,mouthPosition,mouthPronounced,lipSize
0,0.149525,0.478216,0.867824,0.435746
1,0.388697,0.469670,0.216067,0.773844
2,0.264369,0.529973,0.206519,0.436926
3,0.792319,0.267950,-0.066943,0.413451
4,0.526491,0.770739,0.658132,0.375268
...,...,...,...,...
95,0.510697,0.136285,-0.134761,0.507637
96,0.498891,0.603475,0.217395,0.652927
97,0.597692,0.286356,0.351548,0.551172
98,0.473246,0.618666,0.438942,0.527062


In [28]:
test_y

,mouthSize,mouthPosition,mouthPronounced,lipsSize
0,0.241427,0.614988,0.780424,0.520251
1,0.298667,0.704906,0.733134,0.742770
2,0.228592,0.651212,0.317283,0.440955
3,0.481674,0.214611,0.633301,0.222423
4,0.676315,0.793210,0.581524,0.521908
...,...,...,...,...
95,0.549648,0.204712,0.569278,0.322924
96,0.617910,0.485857,0.463260,0.680530
97,0.445559,0.314909,0.787422,0.434210
98,0.660861,0.420872,0.230623,0.670635
